In [ ]:
#new baseline 
data_baseline= data_next_event.copy()

@jit(parallel = True)
def calculator_pos(case):
    res = np.empty(len(case), dtype=object)
    idx = 0
    count=1
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx] == case[idx-1]):
            count+=1
            res[idx] = count
            
        else:
            count=1
            res[idx]=count

        idx+=1
    res[-1]=count+1
    return res

data_baseline["pos"] = calculator_pos(data_baseline['case'].values)

pop=data_baseline.sort_values(by='pos')
pop['eventnum']=pop['event'].map(event_to_num)
pop2=pop.set_index('pos')
pop3=pop[['pos','eventnum']]
pop4=pop3.groupby(['pos', 'eventnum']).apply(pd.DataFrame.mode).reset_index(drop=True)
pop5=pop4.drop_duplicates(subset='pos')
ptenum= dict(zip(pop5.pos, pop5.eventnum))
num_to_event = {value:key for key, value in event_to_num.items()}
data_baseline['predicted_event_num'] = (data_baseline['pos']+1).map(ptenum)
data_baseline['predicted_event']=data_baseline['predicted_event_num'].map(num_to_event)
data_baseline_final=data_baseline.drop(['predicted_event_num'],axis=1)
data_baseline_final['predicted_time']=df_predicted_time['predicted_time']
data_baseline_final

next_task=[]
predicted_event=[]
for event in data_baseline_final['next_task']:
    next_task.append(str(event))
    

for case in data_baseline_final['predicted_event']:
    predicted_event.append(str(case))

accuracy_score(next_task,predicted_event)